In [1]:
import yfinance as yf

msft = yf.Ticker("MSFT")

In [2]:
msft.info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 221000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. The company operates in three segments: Productivity and Business Processes, Intelligent Cloud, and More Personal Computing. The Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, Microsoft Viva, and Skype for Business; Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. The Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related Client Access Licenses; GitHub that provides a collaboration platform and code hosting service for developers; Nuance provides healthcare and enterprise AI solutions; and Azure, a cloud platform. It also offers enterprise

In [3]:
msft.financials

,2022-06-30,2021-06-30,2020-06-30,2019-06-30
Research Development,24512000000.0,20716000000.0,19269000000.0,16876000000.0
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,83716000000.0,71102000000.0,53036000000.0,43688000000.0
Minority Interest,None,None,None,None
Net Income,72738000000.0,61271000000.0,44281000000.0,39240000000.0
Selling General Administrative,27725000000.0,25224000000.0,24709000000.0,23098000000.0
Gross Profit,135620000000.0,115856000000.0,96937000000.0,82933000000.0
Ebit,83383000000.0,69916000000.0,52959000000.0,42959000000.0
Operating Income,83383000000.0,69916000000.0,52959000000.0,42959000000.0
Other Operating Expenses,None,None,None,None


In [4]:
msft.balance_sheet

,2022-06-30,2021-06-30,2020-06-30,2019-06-30
Intangible Assets,1.129800e+10,7.800000e+09,7.038000e+09,7.750000e+09
Total Liab,1.982980e+11,1.917910e+11,1.830070e+11,1.842260e+11
Total Stockholder Equity,1.665420e+11,1.419880e+11,1.183040e+11,1.023300e+11
Other Current Liab,5.938400e+10,5.261200e+10,4.600100e+10,4.586000e+10
Total Assets,3.648400e+11,3.337790e+11,3.013110e+11,2.865560e+11
Common Stock,8.693900e+10,8.311100e+10,8.055200e+10,7.852000e+10
Other Current Assets,1.693200e+10,1.347100e+10,1.151700e+10,1.013300e+10
Retained Earnings,8.428100e+10,5.705500e+10,3.456600e+10,2.415000e+10
Other Liab,3.085300e+10,3.168100e+10,3.449200e+10,3.569900e+10
Good Will,6.752400e+10,4.971100e+10,4.335100e+10,4.202600e+10


In [5]:
msft.cashflow

,2022-06-30,2021-06-30,2020-06-30,2019-06-30
Investments,1.843800e+10,2.876000e+09,6.980000e+09,5.400000e+08
Change To Liabilities,8.052000e+09,7.431000e+09,5.230000e+09,4.694000e+09
Total Cashflows From Investing Activities,-3.031100e+10,-2.757700e+10,-1.222300e+10,-1.577300e+10
Net Borrowings,-9.023000e+09,-3.750000e+09,-5.518000e+09,-4.000000e+09
Total Cash From Financing Activities,-5.887600e+10,-4.848600e+10,-4.603100e+10,-3.688700e+10
Change To Operating Activities,-3.450000e+08,1.160000e+09,-6.730000e+08,-1.542000e+09
Issuance Of Stock,1.841000e+09,1.693000e+09,1.343000e+09,1.142000e+09
Net Income,7.273800e+10,6.127100e+10,4.428100e+10,3.924000e+10
Change In Cash,-2.930000e+08,6.480000e+08,2.220000e+09,-5.900000e+08
Repurchase Of Stock,-3.269600e+10,-2.738500e+10,-2.296800e+10,-1.954300e+10
